# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: Credit risk modelling using Logistic Regression

## Problem Statement

Predict the loan defaulters using a Logistic Regression model on the credit risk data and calculate credit scores

## Learning Objectives

At the end of the mini-project, you will be able to :

* perform data exploration, preprocessing and visualization
* implement Logistic Regression using manual code or using sklearn library
* evaluate the model using appropriate performance metrics
* develop a credit scoring system

## Dataset

The dataset chosen for this mini-project is the [Give Me Some Credit](https://bigml.com/user/jbosca/gallery/dataset/5a7def3d2a83476e09000456#info) dataset which can be used to build models for predicting loan repayment defaulters

#### Datafields

- **SeriousDlqin2yrs:** Person experienced 90 days past due delinquency or worse
- **RevolvingUtilizationOfUnsecuredLines:** Total balance on credit cards and personal lines of credit except real estate and no installment debt like car loans divided by the sum of credit limits
- **age:** Age of borrower in years
- **NumberOfTime30-59DaysPastDueNotWorse:** Number of times borrower has been 30-59 days past due but no worse in the last 2 years.
- **DebtRatio:** Monthly debt payments, alimony,living costs divided by monthy gross income
- **MonthlyIncome:** Monthly income
- **NumberOfOpenCreditLinesAndLoans:** Number of Open loans (installment like car loan or mortgage) and Lines of credit (e.g. credit cards)
- **NumberOfTimes90DaysLate:** Number of times borrower has been 90 days or more past due.
- **NumberRealEstateLoansOrLines:**	Number of mortgage and real estate loans including home equity lines of credit
- **NumberOfTime60-89DaysPastDueNotWorse:**	Number of times borrower has been 60-89 days past due but no worse in the last 2 years.
- **NumberOfDependents:** Number of dependents in family excluding themselves (spouse, children etc.)

## Information

Credit risk arises when a corporate or individual borrower fails to meet their debt obligations. From the lender's perspective, credit risk could disrupt its cash flows or increase collection costs, since the lender may be forced to hire a debt collection agency to enforce the collection. The loss may be partial or complete, where the lender incurs a loss of part of the loan or the entire loan extended to the borrower.

Credit scoring algorithms, which calculate the probability of default, are the best methods that banks use to determine whether or not a loan should be granted.

In order to build a credit scoring system, the following feature transformations are performed:

#### Weight of Evidence and Information value

Logistic regression is a commonly used technique in credit scoring for solving binary classification problems. Prior to model fitting, another iteration of variable selection is valuable to check if the newly WOE transformed variables are still good model candidates. Preferred candidate variables are those with higher information value having a linear relationship with the dependent variable, have good coverage across all categories, have a normal distribution, contain a notable overall contribution, and are relevant to the business.

**Weight of evidence** (WOE) is a powerful tool for feature representation and evaluation in data science. WOE can provide interpret able transformation to both categorical and numerical features. The weight of evidence tells the predictive power of an independent variable in relation to the dependent variable. Since it evolved from credit scoring world, it is generally described as a measure of the separation of good and bad customers. "Bad Customers" refers to the customers who defaulted on a loan. and "Good Customers" refers to the customers who paid back loan. WOE can be calculated using the below formula:

$$WOE = ln \left( \frac{\%   of  Non\_Events}{\%   of  Events} \right)$$

Steps to calculate WOE
* For a continuous variable, split data into 10 parts (or lesser depending on the distribution).
* Calculate the number of events and non-events in each group (bin)
* Calculate the % of events and % of non-events in each group.
* Calculate WOE by taking natural log of division of % of non-events and % of events

**Information value** is one of the most useful technique to select important variables in a predictive model. It helps to rank variables on the basis of their importance. The IV is calculated using the following formula :
$$IV = ∑ (\% of Non\_Events - \% of Events) * WOE$$

Read more about `WOE` and `IV` from the following [link](https://medium.com/@yanhuiliu104/credit-scoring-scorecard-development-process-8554c3492b2b)

## Grading = 10 Points

### Download the dataset

In [ ]:
!wget -qq https://cdn.iisc.talentsprint.com/CDS/MiniProjects/GiveMeSomeCredit.csv
!pip -qq install xverse

### Import Neccesary Packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import seaborn as sns
from matplotlib import pyplot as plt
import math
from xverse.transformer import MonotonicBinning,WOE

### Load the dataset

In [ ]:
train_data = pd.read_csv("GiveMeSomeCredit.csv")
train_data.head(2)

#### Describe the all statistical properties of the train dataset

In [ ]:
train_data[train_data.columns[1:]].describe()

### Pre-processing (1 point)

#### Remove unwanted columns

In [ ]:
train_data.drop("Unnamed: 0",axis=1,inplace=True)

#### Handle the missing data

Find the how many null values in the dataset and fill with mean or remove.

In [ ]:
train_data.isna().sum()

In [ ]:
# Fill the missing values using mean
train_data= train_data.fillna((train_data.mean()))
train_data.isna().sum()

### EDA &  Visualization ( 1 point)

#### Calculate the percentage of the target lebels and visualize with a graph

In [ ]:
total_len = len(train_data['SeriousDlqin2yrs'])
percentage_labels = (train_data['SeriousDlqin2yrs'].value_counts()/total_len)*100
percentage_labels

In [ ]:
sns.countplot(train_data.SeriousDlqin2yrs).set_title('Data Distribution')
ax = plt.gca()
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2., height + 2, '{:.2f}%'.format(100*(height/total_len)), fontsize=14, ha='center', va='bottom')
ax.set_xlabel("Labels for SeriousDlqin2yrs attribute")
ax.set_ylabel("Numbers of records")
plt.show()

#### Plot the distribution of SeriousDlqin2yrs by age

In [ ]:
sns.kdeplot(train_data[train_data["SeriousDlqin2yrs"] == 0]["age"], label="Not in 2 years")
sns.kdeplot(train_data[train_data["SeriousDlqin2yrs"] == 1]["age"], label="In 2 years")
plt.xlabel('Age')
plt.title('Distribuition of Default Rate by Age')
plt.show()

#### Correlation and the heatmap

In [ ]:
train_data[train_data.columns[:]].corr()
sns.heatmap(train_data[train_data.columns[:]].corr(),fmt=".1f")
plt.show()

### Data Engineering (2 points)

#### Weight of Evidence and Information value

* Arrange the binning for each variable with different bins
    * For eg. Age = 49, Age_quantile_range = (48, 56)
* Calculate information value and chooose the best features based on the rules given below

| Information Value |	Variable Predictiveness |
| --- | --- |
| Less than 0.02	|  Not useful for prediction |
| 0.02 to 0.1	| Weak predictive Power |
|  0.1 to 0.3 | Medium predictive Power |
| 0.3 to 0.5 | Strong predictive Power |
| >0.5 | Suspicious Predictive Power |

* Calculate Weight of evidence for the selected variables

Hint: Use [xverse](https://towardsdatascience.com/introducing-xverse-a-python-package-for-feature-selection-and-transformation-17193cdcd067). It is a machine learning Python module in the space of feature engineering, feature transformation and feature selection. It provides pre-built functions for the above steps, such as binning and conversion to WoE.

In [ ]:
# Using xverse package
clf = MonotonicBinning()
clf.fit(train_data.iloc[:,1:], train_data.iloc[:,0])
out_X = clf.transform(train_data.iloc[:,1:])
out_X.head()

In [ ]:
# Using xverse package
clf = WOE()
clf.fit(train_data.iloc[:,1:], train_data.iloc[:,0])
out_X = clf.transform(train_data.iloc[:,1:])

In [ ]:
clf.iv_df #information value dataset

In [ ]:
selected_columns = clf.iv_df.Variable_Name.values[:5]

### Identify features,  target and split it into train and test

In [ ]:
len(out_X.columns), len(selected_columns)

In [ ]:
train_data.iloc[:,1:]

In [ ]:
X = out_X[selected_columns]
y = train_data['SeriousDlqin2yrs']
X.shape, y.shape

In [ ]:
# split the data into train and test
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=66)
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

### Logistic Regression from scratch using gradient method (2 points)

For Linear Regression, we had the hypothesis $yhat = w.X +b$ , whose output range was the set of all Real Numbers.
Now, for Logistic Regression our hypothesis is  $yhat = sigmoid(w.X + b)$ , whose output range is between 0 and 1 because by applying a sigmoid function, we always output a number between 0 and 1.

$yhat = \frac{1}{1 +e^{-(w.x+b)}}$

Hint: [logistic-regression-with-python](
https://medium.com/@ODSC/logistic-regression-with-python-ede39f8573c7)

In [ ]:
intercept = np.ones((xtrain.shape[0], 1))
x_train = np.concatenate((intercept, xtrain), axis=1)
weight = np.zeros(x_train.shape[1])

In [ ]:
weight.shape

In [ ]:
#Sigmoid method
def sigmoid(x, weight):
    z = np.dot(x, weight)
    return 1 / (1 + np.exp(-z))

from sklearn.metrics import mean_squared_error

def fit(x, y, weight, lr, iterations):
    for i in range(iterations):
        sigma = sigmoid(x, weight)
        loss = mean_squared_error(sigma,y)
        # gradient
        dW = np.dot(x.T, (sigma - y)) / y.shape[0]
        #Updating the weights
        weight -= lr * dW
    return weight

In [ ]:
#creating the class Object
updated_weights = fit(x_train, ytrain, weight, 0.1 , 5000)

In [ ]:
# Method to predict the class label.
def predict(x_new , weight, treshold):
    x_new = np.concatenate((np.ones((x_new.shape[0], 1)), x_new), axis=1)
    result = sigmoid(x_new, weight)
    result = result >= treshold
    y_pred = np.zeros(result.shape[0])
    for i in range(len(y_pred)):
        if result[i] == True:
            y_pred[i] = 1
        else:
            continue
    return y_pred

In [ ]:
xtest.shape

In [ ]:
y_pred = predict(xtest, updated_weights, 0.5)

In [ ]:
# Accuracy of test data
(y_pred == ytest).sum() / len(y_pred)

### Implement the Logistic regression using sklearn (2 points)

As there is imbalance in the class distribution, add weightage to the Logistic regression.

* Find the accuracy with class weightage in Logistic regression
* Find the accuracy without class weightage in Logistic regression

Hint: [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [ ]:
# With weightage
log_reg = LogisticRegression(class_weight={0:6/100, 1: 94/100})
log_reg.fit(xtrain,ytrain)
log_reg.score(xtest,ytest), log_reg.score(xtrain, ytrain)

In [ ]:
test_predicted = log_reg.predict(xtest)

In [ ]:
# Without weightage
log_reg = LogisticRegression()
log_reg.fit(xtrain,ytrain)
log_reg.score(xtest,ytest), log_reg.score(xtrain, ytrain)

In [ ]:
test_predicted = log_reg.predict(xtest)

In [ ]:
log_reg.coef_, log_reg.intercept_

In [ ]:
log_reg.predict_proba

### Credit scoring (1 point)

When scaling the model into a scorecard, we will need both the Logistic Regression coefficients from model fitting as well as the transformed WoE values. We will also need to convert the score from the model from the log-odds unit to a points system.
For each independent variable Xi, its corresponding score is:

$Score = \sum_{i=1}^{n} (-(β_i × WoE_i + \frac{α}{n}) × Factor + \frac{Offset}{n})$

Where:

βi — logistic regression coefficient for the variable Xi

α — logistic regression intercept

WoE — Weight of Evidence value for variable Xi

n — number of independent variable Xi in the model

Factor, Offset — known as scaling parameter

  - Factor = pdo / ln(2); pdo is points to double the odds
  - Offset = Round_of_Score - {Factor * ln(Odds)}

In [ ]:
coef = log_reg.coef_.ravel()
intercept = log_reg.intercept_
factor = 20/np.log(2)
offset = 600 - ( factor * np.log(50))
factor, offset

In [ ]:
# 1st method
# all_scores = []
# for idx,row in X.iterrows():
#   score  = []
#   for j in range(len(row)):
#     asum = (-((row[j] * coef[j]) + (intercept/X.shape[1])) * factor) + (offset/X.shape[1])
#     score.append(asum)
#   all_scores.append(sum(score))
# max(all_scores), min(all_scores)

In [ ]:
# 2nd method
all_scores = []
for idx,row in X.iterrows():
  a = row.values * coef          # B_i * WOE_i
  a = a + (intercept/X.shape[1]) # (B_i * WOE_i) + intercept_i / n
  b = -a * factor                # -((B_i * WOE_i) + intercept_i / n) * factor
  b = b + (offset/X.shape[1])    # -((B_i * WOE_i) + intercept_i / n) * factor) + offset / n
  all_scores.append(sum(b))      # sum

In [ ]:
max(all_scores),min(all_scores)

### Performance Metrics (1 point)

#### Precision

In [ ]:
from sklearn.metrics import precision_score
precision_score(ytest, test_predicted ,average='macro')

#### Recall

In [ ]:
from sklearn.metrics import recall_score
recall_score(ytest, test_predicted,average='macro')

#### Classification Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, test_predicted))

#### Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(ytest, test_predicted)
mat